In [2]:
!pip install pulp


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.7/17.7 MB 93.2 MB/s eta 0:00:00


In [3]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum
import random

# 轉移數量 T 代表 T 個轉移，對應 T+1 個 token (包含 START 和 STOP)
T = 5

# 定義普通標籤
normal_labels = ['O', 'B', 'I']
num_normal = len(normal_labels)

# 定義特殊標籤
start_token = 'START'
stop_token = 'STOP'

# 取得普通標籤的索引
label_to_index = {label: idx for idx, label in enumerate(normal_labels)}
B_idx = label_to_index['B']
O_idx = label_to_index['O']
I_idx = label_to_index['I']

# 定義 ILP 問題
model = LpProblem("sequence-labeling", LpMaximize)

# 定義變數 x
# t = 0: 從 START 轉移到某個普通標籤
x = {}
for j in range(num_normal):
    x[0, start_token, j] = LpVariable(f"x_0_START_{j}", cat="Binary")

# t = 1 到 T-2: 普通標籤之間的轉移
for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            x[t, i, j] = LpVariable(f"x_{t}_{i}_{j}", cat="Binary")

# t = T-1: 從某個普通標籤轉移到 STOP
for i in range(num_normal):
    x[T-1, i, stop_token] = LpVariable(f"x_{T-1}_{i}_STOP", cat="Binary")

# 隨機初始化轉移分數
s = {}
# t = 0: START -> 普通標籤
for j in range(num_normal):
    s[0, start_token, j] = random.uniform(-1, 1)
# t = 1 到 T-2: 普通標籤之間的轉移
for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            s[t, i, j] = random.uniform(-1, 1)
# t = T-1: 普通標籤 -> STOP
for i in range(num_normal):
    s[T-1, i, stop_token] = random.uniform(-1, 1)

# 目標函數：最大化總得分
model += lpSum(s[key] * x[key] for key in x)

# 約束：每個時間步只能選擇一個轉移
# t = 0
model += lpSum(x[0, start_token, j] for j in range(num_normal)) == 1
# t = 1 到 T-2
for t in range(1, T-1):
    model += lpSum(x[t, i, j] for i in range(num_normal) for j in range(num_normal)) == 1
# t = T-1
model += lpSum(x[T-1, i, stop_token] for i in range(num_normal)) == 1

# 連續性約束：
# (1) t = 0 到 t = 1：第一個轉移的目的標籤應成為下一步的來源標籤
for j in range(num_normal):
    model += x[0, start_token, j] == lpSum(x[1, j, k] for k in range(num_normal))
# (2) t = 1 到 t = T-2：對每個普通標籤，前一轉移的目的等於下一轉移的來源
for t in range(1, T-2):
    for label in range(num_normal):
        model += lpSum(x[t, i, label] for i in range(num_normal)) == lpSum(x[t+1, label, j] for j in range(num_normal))
# (3) t = T-2 到 t = T-1：中間步驟的目的標籤等於最後步驟轉移中的來源標籤
for label in range(num_normal):
    model += lpSum(x[T-2, i, label] for i in range(num_normal)) == x[T-1, label, stop_token]

# BIO 規則：
# (a) I 不可跟在 O 之後：適用於普通標籤之間的轉移 (t = 1 到 T-2)
for t in range(1, T-1):
    model += x[t, O_idx, I_idx] == 0

# (b) 第一個轉移（t = 0）的目的標籤必須是 B
for j in range(num_normal):
    if j != B_idx:
        model += x[0, start_token, j] == 0

# 求解 ILP
model.solve()

print("🔹 Optimal Sequence Labeling Path:")
# t = 0: START -> 普通標籤
for j in range(num_normal):
    if x[0, start_token, j].value() == 1:
        print(f"Time step 0: {start_token} -> {normal_labels[j]}")
# t = 1 到 T-2：普通標籤之間的轉移
for t in range(1, T-1):
    for i in range(num_normal):
        for j in range(num_normal):
            if x[t, i, j].value() == 1:
                print(f"Time step {t}: {normal_labels[i]} -> {normal_labels[j]}")
# t = T-1: 普通標籤 -> STOP
for i in range(num_normal):
    if x[T-1, i, stop_token].value() == 1:
        print(f"Time step {T-1}: {normal_labels[i]} -> {stop_token}")


🔹 Optimal Sequence Labeling Path:
Time step 0: START -> B
Time step 1: B -> B
Time step 2: B -> I
Time step 3: I -> O
Time step 4: O -> STOP


In [ ]:
from pulp import LpMaximize, LpProblem, LpVariable, lpSum, LpBinary
import random

# 設定句子長度 (n=5 表示有 5 個非根節點: w1, ..., w5)
n = 5
words = ["$", "w1", "w2", "w3", "w4", "w5"]

# 建立 ILP 模型 (最大化問題)
model = LpProblem("DependencyParsing", LpMaximize)

# 定義變數：x[i,j] 表示 w_i 為 w_j 的父節點
# i 範圍從 0 (根節點) 到 n，j 範圍從 1 到 n (非根節點)
x = {}
for i in range(n+1):
    for j in range(1, n+1):
        x[(i, j)] = LpVariable(f"x_{i}_{j}", cat=LpBinary)

# 禁止自環：對於每個非根節點 w_j, 令 x[j,j] = 0
for j in range(1, n+1):
    model += x[(j, j)] == 0

# 每個非根節點必須恰好有一個父節點：對 j=1,...,n
for j in range(1, n+1):
    model += lpSum(x[(i, j)] for i in range(n+1)) == 1

# 假設 score(i,j) 為轉移分數，這裡用隨機分數示例
score = {}
for i in range(n+1):
    for j in range(1, n+1):
        score[(i, j)] = random.uniform(-1, 1)

# 目標函數：最大化所有轉移分數之和
model += lpSum(score[(i, j)] * x[(i, j)] for i in range(n+1) for j in range(1, n+1))

# 可選：如果需要投射性約束 (Projectivity)，可加入以下約束：
for i in range(n+1):
    for j in range(i+1, n+1):
        if j - i - 1 > 0:
            model += (j - i - 1) * x[(i, j)] <= lpSum(x[(p, k)] for k in range(i+1, j) for p in range(i, j+1))

# 求解 ILP 模型
model.solve()

# 輸出依存解析結果
print("Dependency Parsing Tree:")
for j in range(1, n+1):
    for i in range(n+1):
        if x[(i, j)].varValue == 1:
            print(f"Parent of {words[j]} is {words[i]}")


Dependency Parsing Tree:
Parent of w1 is $
Parent of w2 is $
Parent of w3 is $
Parent of w4 is w5
Parent of w5 is w3
